In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
import gensim
import string

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.model_selection import train_test_split


In [ ]:
stop_factory = StopWordRemoverFactory()
more_stopword = ['dengan', 'ia','bahwa','oleh']
data = stop_factory.get_stop_words()+more_stopword
stopword = stop_factory.create_stop_word_remover()

Fact = StemmerFactory()
stemmer = Fact.create_stemmer()
# lemmatizer = WordNetLemmatizer()

In [ ]:
cwd = os.getcwd()
data_path = os.path.join(cwd, 'dataset')

files = []
for file in os.listdir(data_path):
    if file.endswith('.csv'):
        files.append(os.path.join(data_path, file))
        
print(files, len(files))

In [ ]:
raw_data1 = pd.read_csv(files[1], sep = ';', encoding_errors = 'ignore')
raw_data2 = pd.read_csv(files[2], sep = ';', encoding_errors = 'ignore')
raw_data3 = pd.read_csv(files[4], sep = ';', encoding_errors = 'ignore')

In [ ]:
raw_data1.sample(5)

In [ ]:
raw_data2.sample(5)

In [ ]:
raw_data3.sample(5)

In [ ]:
data3 = raw_data3.drop(columns = raw_data3.columns[3:])
data3.sample(5)

In [ ]:
data1 = raw_data1[['berita', 'kategori']]
data1.rename(columns = {'kategori' : 'label'}, inplace = True)

data2 = raw_data2.rename(columns = {'tagging' : 'label'})

data3 = raw_data3[['Body', 'Label']]
data3.rename(columns = {'Label' : 'label', 'Body' : 'berita'}, inplace = True)

data1.columns, data2.columns, data3.columns


In [ ]:
df = pd.concat([data1, data2, data3], ignore_index = True)
df.sample(5)

In [ ]:
df.label.unique()

In [ ]:
df.label = df.label.replace(['valid', 'hoax', 'Valid', 'Hoax'], [0, 1, 0, 1])
df.label.unique()

In [ ]:
df.head()

In [ ]:
df.label.value_counts().plot(kind = 'bar')

In [ ]:
df[df.duplicated()]

In [ ]:
df = df.drop_duplicates()
df[df.duplicated()]

In [ ]:
df.shape

In [ ]:
df.berita[0]

In [ ]:
archive_path = os.path.join(data_path, 'archive', 'Summarized')
tempo_sum = pd.read_excel(os.path.join(archive_path, 'dataset_tempo_summarized.xlsx'), index_col = 'index')
cnn_sum = pd.read_excel(os.path.join(archive_path, 'dataset_cnn_summarized.xlsx'), index_col = 'index')
kompas_sum = pd.read_excel(os.path.join(archive_path, 'dataset_kompas_summarized.xlsx'), index_col = 'index')
tbhoax_sum = pd.read_excel(os.path.join(archive_path, 'dataset_turnbackhoax_summarized.xlsx'), index_col = 'index')

In [ ]:
tempo_sum.sample(5)

In [ ]:
cnn_sum.sample(5)

In [ ]:
kompas_sum.sample(5)

In [ ]:
tbhoax_sum.sample(5)

In [ ]:
def pipeline(text):
    text = re.sub(r'http\S+', '', text.lower())
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans("","",string.punctuation))
#     text = tokenizer(text)
    text = stemmer.stem(text)
    text = stopword.remove(text)
    return text.split()

pipeline(cnn_sum.cleaned[0])

In [ ]:
X_processed = X.apply(lambda x : pipeline(x))

df_processed = pd.DataFrame({
    'berita' : X_processed,
    'label' : y
})

# df_processed.to_csv(os.path.join(data_path, 'df_processed.csv'))